In [28]:
%cd "C:\Users\jigna\PredictiveAnalyticsProject"

C:\Users\jigna\PredictiveAnalyticsProject


In [29]:
import pandas as pd

# Import dataset
data = pd.read_csv('data.csv')

In [30]:
# Check for missing values
data.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64

In [31]:
data.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
deposit      object
dtype: object

In [33]:
# Use sklearn to create train and test data sets

from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

y_train = data['deposit']
X_train = data.drop(columns=['deposit'])

# Initialize lists of string anf numeric variables
str_vars = [] # String variables
num_vars = [] # Numeric variables

# Loop through data.columns and append the column to the apprpriate str_vars or num_vars
# if the conditions are met
for i in X_train.columns:
    if X_train.dtypes[i] == 'object':
        str_vars.append(i)
    else:
        num_vars.append(i)
        
# Update X_train with data from the imputer
ct = make_column_transformer(
    (OneHotEncoder(sparse=False), str_vars),
    (StandardScaler(), num_vars)
)
ct.fit(X_train)
X_train = pd.DataFrame(ct.transform(X_train), columns=ct.get_feature_names_out())
X_train

# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

,onehotencoder__job_admin.,onehotencoder__job_blue-collar,onehotencoder__job_entrepreneur,onehotencoder__job_housemaid,onehotencoder__job_management,onehotencoder__job_retired,onehotencoder__job_self-employed,onehotencoder__job_services,onehotencoder__job_student,onehotencoder__job_technician,...,onehotencoder__poutcome_other,onehotencoder__poutcome_success,onehotencoder__poutcome_unknown,standardscaler__age,standardscaler__balance,standardscaler__day,standardscaler__duration,standardscaler__campaign,standardscaler__pdays,standardscaler__previous
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,-0.270775,-0.431243,-0.188961,-0.749666,-0.553282,0.000821,0.508099
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,-0.016722,-0.357448,-0.902715,-0.490065,0.909843,-0.474473,-0.355777
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-0.186091,0.290280,-1.378550,0.989660,0.178281,1.985872,0.508099
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.660750,1.456439,-0.545838,0.507956,-0.553282,-0.474473,-0.355777
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,-0.355459,0.054630,1.595422,0.089710,-0.187501,1.874038,2.235851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8366,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.914803,-0.350316,1.119586,-0.291038,-0.553282,-0.474473,-0.355777
8367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,-0.186091,0.384540,-1.378550,0.822362,0.178281,-0.474473,-0.355777
8368,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,-0.270775,-0.415430,0.524792,-0.368918,-0.553282,-0.474473,-0.355777
8369,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.507591,-0.485815,-0.307920,-0.360265,-0.187501,-0.474473,-0.355777


In [35]:
# Lasso regression model

import numpy as np
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

# Update X_train with data from the imputer
imp = IterativeImputer(add_indicator=True)
imp.fit(X_train)
orig_columns = X_train.columns
X_train = pd.DataFrame(imp.transform(X_train), columns=imp.get_feature_names_out())

# Update original columns of X_train using the StandardScaler
ct = make_column_transformer(
    (StandardScaler(), orig_columns),
    remainder='passthrough'
)
X_train = pd.DataFrame(ct.fit_transform(X_train), columns=ct.get_feature_names_out())

# Having used cross-validation to determine lasso regression with alpha=.05 generalizes well,
# let's train the model on the entire X_train data set and test it with the X_test data set

# Create linear regression object
regr = Lasso(alpha=.05)

# Train the model using the training sets
regr.fit(X_train, y_train)

print("Training set score: {:.3f}".format(regr.score(X_train, y_train)))

X_test = pd.DataFrame(imp.transform(X_test), columns=imp.get_feature_names_out())
X_test = pd.DataFrame(ct.transform(X_test), columns=ct.get_feature_names_out())
print("Test set score: {:.3f}".format(regr.score(X_test, y_test)))

AttributeError: 'IterativeImputer' object has no attribute 'get_feature_names_out'